# **IPL Win Probability Predictor**

---
This is the data of IPL from 2008 to 2019 and we have to find the probability of winning and loosing between two teams, this is the problem of classification but we want the probability so we are using only Logistic regression and support vector machine

In [1]:
#importing required labraries
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

## **Data Collection**

In [2]:
match = pd.read_csv('matches.csv')
delivery = pd.read_csv('deliveries.csv')

In [3]:
match.head() # all matches of IPL 

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [4]:
match.shape

(756, 18)

In [5]:
delivery.head() # ball by ball details of each match

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


**we did't want all this data we only want "batting_team", "bowling_team", "city", "runs_left", "balls_left", "total_runs_x", "crr", "rrr", "result" thus features.**


In [6]:
# to get score of 1st inning and 2nd inning
total_score_df = delivery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()#using reset index converted to dataset

C:\Users\Tushar\AppData\Local\Temp\ipykernel_12680\1433432954.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_score_df = delivery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()#using reset index converted to dataset


In [7]:
# we only want first inning score so here we only get first inning data using filter on "total_score_df"
total_score_df = total_score_df[total_score_df['inning'] == 1]

In [8]:
total_score_df

,match_id,inning,total_runs
0,1,1,207
2,2,1,184
4,3,1,183
6,4,1,163
8,5,1,157
...,...,...,...
1518,11347,1,143
1520,11412,1,136
1522,11413,1,171
1524,11414,1,155


In [9]:
# now we have to merge with matches dataset
match_df = match.merge(total_score_df[['match_id','total_runs']],left_on='id',right_on='match_id')
# left_on and right_on are the joining conditions

In [10]:
match_df # in this dataset we get new column which have 1st inning score of each match

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3,match_id,total_runs
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN,1,207
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN,2,184
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN,3,183
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN,4,163
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN,5,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,11347,IPL-2019,Mumbai,05-05-2019,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,9,HH Pandya,Wankhede Stadium,Nanda Kishore,O Nandan,S Ravi,11347,143
752,11412,IPL-2019,Chennai,07-05-2019,Chennai Super Kings,Mumbai Indians,Chennai Super Kings,bat,normal,0,Mumbai Indians,0,6,AS Yadav,M. A. Chidambaram Stadium,Nigel Llong,Nitin Menon,Ian Gould,11412,136
753,11413,IPL-2019,Visakhapatnam,08-05-2019,Sunrisers Hyderabad,Delhi Capitals,Delhi Capitals,field,normal,0,Delhi Capitals,0,2,RR Pant,ACA-VDCA Stadium,NaN,NaN,NaN,11413,171
754,11414,IPL-2019,Visakhapatnam,10-05-2019,Delhi Capitals,Chennai Super Kings,Chennai Super Kings,field,normal,0,Chennai Super Kings,0,6,F du Plessis,ACA-VDCA Stadium,Sundaram Ravi,Bruce Oxenford,Chettithody Shamshuddin,11414,155


In [11]:
match_df['team1'].unique() # all teams from dataset

array(['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
       'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Delhi Capitals'], dtype=object)

"Gujarat Lions" ,"Rising Pune Supergiant","Kochi Tuskers Kerala", "Pune Warriors" ,"Rising Pune Supergiants" now thus teams are not existing so we will drop all columns and we combine "Delhi Daredevils" and "Delhi Capitals" 

"Deccan Chargers" are converted to "Sunrisers Hyderabad"

In [12]:
teams = [
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders',
    'Kings XI Punjab',
    'Chennai Super Kings',
    'Rajasthan Royals',
    'Delhi Capitals'
]

In [13]:
match_df['team1'] = match_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team2'] = match_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['team1'] = match_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2'] = match_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [14]:
match_df = match_df[match_df['team1'].isin(teams)] # now here we only get thus teams who are mentioned above
match_df = match_df[match_df['team2'].isin(teams)]

In [15]:
match_df.shape # now this is our dataset

(641, 20)

In [16]:
match_df['dl_applied'].value_counts() # we did not  want this 15  matches because they are creating some noise in data

0    626
1     15
Name: dl_applied, dtype: int64

In [17]:
match_df = match_df[match_df['dl_applied'] == 0] 

so till now we done some data cleaning and extraction where we created 1st inningscore column and keept only thus teams who are playing right now and we deleted thus matches where rain is occurs

In [18]:
match_df = match_df[['match_id','city','winner','total_runs']] # thus column we want from match_df

In [19]:
delivery_df = match_df.merge(delivery,on='match_id') # two dataframe are joined on bases of match_id

In [20]:
delivery_df = delivery_df[delivery_df['inning'] == 2] # only thus rows who have inning == 2

In [21]:
delivery_df

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,0,0,0,1,0,1,NaN,NaN,NaN
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,0,0,0,NaN,NaN,NaN
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,0,0,0,0,0,0,NaN,NaN,NaN
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,0,0,0,0,2,0,2,NaN,NaN,NaN
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,0,0,0,0,4,0,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149573,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,...,0,0,0,0,1,0,1,NaN,NaN,NaN
149574,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,...,0,0,0,0,2,0,2,NaN,NaN,NaN
149575,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,...,0,0,0,0,1,0,1,SR Watson,run out,KH Pandya
149576,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,...,0,0,0,0,2,0,2,NaN,NaN,NaN


In delivery_df there is column total_runs_y who tell how many runs are made on that perticular ball, now we have to calculate for every match how much score are after ball 

In [22]:
delivery_df['current_score'] = delivery_df.groupby('match_id').cumsum()['total_runs_y']

C:\Users\Tushar\AppData\Local\Temp\ipykernel_12680\3025607697.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  delivery_df['current_score'] = delivery_df.groupby('match_id').cumsum()['total_runs_y']


In [23]:
# runs left calculated
delivery_df['runs_left'] = delivery_df['total_runs_x'] - delivery_df['current_score']

In [24]:
# balls left calculated
delivery_df['balls_left'] = 126 - (delivery_df['over']*6 + delivery_df['ball'])

In [25]:
delivery_df

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_left,balls_left
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,1,0,1,NaN,NaN,NaN,1,206,119
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,NaN,NaN,NaN,1,206,118
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,0,0,0,NaN,NaN,NaN,1,206,117
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,0,2,0,2,NaN,NaN,NaN,3,204,116
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,0,4,0,4,NaN,NaN,NaN,7,200,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149573,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,...,0,1,0,1,NaN,NaN,NaN,152,0,4
149574,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,...,0,2,0,2,NaN,NaN,NaN,154,-2,3
149575,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,...,0,1,0,1,SR Watson,run out,KH Pandya,155,-3,2
149576,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,...,0,2,0,2,NaN,NaN,NaN,157,-5,1


In [26]:
# from this we calculated how many wickets left
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].fillna("0")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].apply(lambda x:x if x == "0" else "1") #NaN became 0 and if nameis there then 1
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].astype('int') # converted in int
wickets = delivery_df.groupby('match_id').cumsum()['player_dismissed'].values # bases of match_id done group by
delivery_df['wickets'] = 10 - wickets
delivery_df.head()

C:\Users\Tushar\AppData\Local\Temp\ipykernel_12680\2925096708.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  wickets = delivery_df.groupby('match_id').cumsum()['player_dismissed'].values # bases of match_id done group by


,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_left,balls_left,wickets
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,1,0,1,0,NaN,NaN,1,206,119,10
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,NaN,NaN,1,206,118,10
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,0,0,0,NaN,NaN,1,206,117,10
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,2,0,2,0,NaN,NaN,3,204,116,10
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,4,0,4,0,NaN,NaN,7,200,115,10


In [27]:
# crr = runs/overs formula for current runrate
delivery_df['crr'] = (delivery_df['current_score']*6)/(120 - delivery_df['balls_left'])

In [28]:
# to calculate required run rate
delivery_df['rrr'] = (delivery_df['runs_left']*6)/delivery_df['balls_left']

In [29]:
# this function is to display result
def result(row):
    return 1 if row['batting_team'] == row['winner'] else 0

In [30]:
delivery_df['result'] = delivery_df.apply(result,axis=1) # above function is applyed here

In [31]:
# taken thus columns who required for prediction
final_df = delivery_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets','total_runs_x','crr','rrr','result']]

In [33]:
# here we do shuffling to avoid bias
final_df = final_df.sample(final_df.shape[0])

In [34]:
# show any random row
final_df.sample()

,batting_team,bowling_team,city,runs_left,balls_left,wickets,total_runs_x,crr,rrr,result
76366,Delhi Daredevils,Chennai Super Kings,Delhi,136,81,6,169,5.076923,10.074074,0


In [36]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72413 entries, 138112 to 1876
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   batting_team  72413 non-null  object 
 1   bowling_team  72413 non-null  object 
 2   city          71581 non-null  object 
 3   runs_left     72413 non-null  int64  
 4   balls_left    72413 non-null  int64  
 5   wickets       72413 non-null  int32  
 6   total_runs_x  72413 non-null  int64  
 7   crr           72413 non-null  float64
 8   rrr           72406 non-null  float64
 9   result        72413 non-null  int64  
dtypes: float64(2), int32(1), int64(4), object(3)
memory usage: 5.8+ MB


we perform one hot encoding on object datatype column

In [39]:
final_df = pd.get_dummies(final_df,columns=['batting_team','bowling_team','city'])

In [40]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72413 entries, 138112 to 1876
Data columns (total 56 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   runs_left                                 72413 non-null  int64  
 1   balls_left                                72413 non-null  int64  
 2   wickets                                   72413 non-null  int32  
 3   total_runs_x                              72413 non-null  int64  
 4   crr                                       72413 non-null  float64
 5   rrr                                       72406 non-null  float64
 6   result                                    72413 non-null  int64  
 7   batting_team_Chennai Super Kings          72413 non-null  uint8  
 8   batting_team_Deccan Chargers              72413 non-null  uint8  
 9   batting_team_Delhi Capitals               72413 non-null  uint8  
 10  batting_team_Delhi Daredevils 

**lets check for null values**

In [41]:
final_df.isna().sum()

runs_left                                   0
balls_left                                  0
wickets                                     0
total_runs_x                                0
crr                                         0
rrr                                         7
result                                      0
batting_team_Chennai Super Kings            0
batting_team_Deccan Chargers                0
batting_team_Delhi Capitals                 0
batting_team_Delhi Daredevils               0
batting_team_Kings XI Punjab                0
batting_team_Kolkata Knight Riders          0
batting_team_Mumbai Indians                 0
batting_team_Rajasthan Royals               0
batting_team_Royal Challengers Bangalore    0
batting_team_Sunrisers Hyderabad            0
bowling_team_Chennai Super Kings            0
bowling_team_Deccan Chargers                0
bowling_team_Delhi Capitals                 0
bowling_team_Delhi Daredevils               0
bowling_team_Kings XI Punjab      

In [42]:
# drop all null values
final_df.dropna(inplace=True)

In [44]:
final_df.isna().sum()

runs_left                                   0
balls_left                                  0
wickets                                     0
total_runs_x                                0
crr                                         0
rrr                                         0
result                                      0
batting_team_Chennai Super Kings            0
batting_team_Deccan Chargers                0
batting_team_Delhi Capitals                 0
batting_team_Delhi Daredevils               0
batting_team_Kings XI Punjab                0
batting_team_Kolkata Knight Riders          0
batting_team_Mumbai Indians                 0
batting_team_Rajasthan Royals               0
batting_team_Royal Challengers Bangalore    0
batting_team_Sunrisers Hyderabad            0
bowling_team_Chennai Super Kings            0
bowling_team_Deccan Chargers                0
bowling_team_Delhi Capitals                 0
bowling_team_Delhi Daredevils               0
bowling_team_Kings XI Punjab      

In [46]:
final_df.describe()

,runs_left,balls_left,wickets,total_runs_x,crr,rrr,result,batting_team_Chennai Super Kings,batting_team_Deccan Chargers,batting_team_Delhi Capitals,...,city_Kolkata,city_Mohali,city_Mumbai,city_Nagpur,city_Port Elizabeth,city_Pune,city_Raipur,city_Ranchi,city_Sharjah,city_Visakhapatnam
count,72406.000000,72406.000000,72406.000000,72406.000000,72406.000000,7.240600e+04,72406.000000,72406.000000,72406.000000,72406.000000,...,72406.000000,72406.000000,72406.000000,72406.000000,72406.000000,72406.000000,72406.000000,72406.000000,72406.000000,72406.000000
mean,92.267298,62.663136,7.536351,165.583695,7.439429,NaN,0.525606,0.117186,0.050175,0.016090,...,0.107781,0.016601,0.136867,0.005138,0.011145,0.018244,0.007817,0.009820,0.009599,0.014308
std,50.016154,33.400526,2.138373,29.282132,2.275892,NaN,0.499347,0.321645,0.218308,0.125822,...,0.310106,0.127771,0.343709,0.071494,0.104983,0.133835,0.088068,0.098607,0.097502,0.118759
min,-16.000000,-2.000000,0.000000,65.000000,0.000000,-inf,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,53.000000,35.000000,6.000000,146.000000,6.257143,7.142857e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,92.000000,63.000000,8.000000,165.000000,7.480519,8.877551e+00,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,130.000000,92.000000,9.000000,185.000000,8.682353,1.090909e+01,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,249.000000,119.000000,10.000000,250.000000,42.000000,inf,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


in rrr we get infinite values so we will drop that row who have 0 in balls_left column

In [47]:
final_df = final_df[final_df['balls_left'] != 0]

### **Train test split**

In [48]:
from sklearn.model_selection import train_test_split
x = final_df.drop('result',axis=1)
y = final_df['result']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

### **Logistic Regression**

In [50]:
from sklearn.linear_model import LogisticRegression #imported Logistic Regression model
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,precision_score,recall_score,f1_score

logistic_model = LogisticRegression() # Logistic Regression instance created
logistic_model.fit(x_train,y_train) # model fitting

# Model evaluation
y_pred_test = logistic_model.predict(x_test)
cnf_matrix = confusion_matrix(y_test,y_pred_test)
print(f"Test data confusion matrix : \n{cnf_matrix}")
Logistic_Regression_Accuracy = accuracy_score(y_test,y_pred_test)
print(f"Test data accuracy score : {Logistic_Regression_Accuracy}")
print('*'*70)
y_pred_train = logistic_model.predict(x_train)
cnf_matrix = confusion_matrix(y_train,y_pred_train)
print(f"Train data confusion matrix : \n{cnf_matrix}")
acc = accuracy_score(y_train,y_pred_train)
print(f"Train data accuracy score : {acc}")

Test data confusion matrix : 
[[5291 1556]
 [1387 6201]]
Test data accuracy score : 0.7961205403533079
**********************************************************************
Train data confusion matrix : 
[[21302  5989]
 [ 5624 24822]]
Train data accuracy score : 0.7988638134991427


c:\Users\Tushar\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


we want probability so we used logistic regression

In [52]:
logistic_model.predict_proba(x_test)[2] # this is how we get probability

array([0.54043049, 0.45956951])

In [56]:
x_test.head(2)

,runs_left,balls_left,wickets,total_runs_x,crr,rrr,batting_team_Chennai Super Kings,batting_team_Deccan Chargers,batting_team_Delhi Capitals,batting_team_Delhi Daredevils,...,city_Kolkata,city_Mohali,city_Mumbai,city_Nagpur,city_Port Elizabeth,city_Pune,city_Raipur,city_Ranchi,city_Sharjah,city_Visakhapatnam
31111,51,24,4,150,6.187500,12.750000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41341,66,17,3,178,6.524272,23.294118,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [57]:
teams

['Sunrisers Hyderabad',
 'Mumbai Indians',
 'Royal Challengers Bangalore',
 'Kolkata Knight Riders',
 'Kings XI Punjab',
 'Chennai Super Kings',
 'Rajasthan Royals',
 'Delhi Capitals']

In [58]:
delivery_df['city'].unique()

array(['Hyderabad', 'Bangalore', 'Mumbai', 'Indore', 'Kolkata', 'Delhi',
       'Chandigarh', 'Jaipur', 'Chennai', 'Cape Town', 'Port Elizabeth',
       'Durban', 'Centurion', 'East London', 'Johannesburg', 'Kimberley',
       'Bloemfontein', 'Ahmedabad', 'Cuttack', 'Nagpur', 'Dharamsala',
       'Visakhapatnam', 'Pune', 'Raipur', 'Ranchi', 'Abu Dhabi',
       'Sharjah', nan, 'Mohali', 'Bengaluru'], dtype=object)

In [87]:
batting_team = 'Delhi Capitals'
bowling_team =	'Chennai Super Kings'
city =	'Delhi'
runs_left = 10	
balls_left = 81	
wickets = 6
total_runs_x = 169	
crr	= 5.07
rrr	= 10.07

column_names = x.columns
test_array = np.zeros(len(x.columns))
batting_team_index = np.where(column_names == "batting_team_"+batting_team)[0][0] 
bowling_team_index = np.where(column_names == "bowling_team_"+bowling_team)[0][0]
city_index = np.where(column_names == "city_"+city)[0][0]

test_array[batting_team_index] = 1
test_array[bowling_team_index] = 1
test_array[city_index] = 1
test_array[0] = runs_left
test_array[1] = balls_left
test_array[2] = wickets
test_array[3] = total_runs_x
test_array[4] = crr
test_array[5] = rrr

array = logistic_model.predict_proba([test_array])

print(f"Probability of winning of {batting_team} is {array[0][1]}")
print(f"Probability of winning of {bowling_team} is {array[0][0]}")


Probability of winning of Delhi Capitals is 0.9454107445593452
Probability of winning of Chennai Super Kings is 0.05458925544065485


c:\Users\Tushar\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [76]:
column_names = x.columns
test_array = np.zeros(len(x.columns))
batting_team_index = np.where(column_names == "batting_team_"+batting_team)[0][0] 
bowling_team_index = np.where(column_names == "bowling_team_"+bowling_team)[0][0]
city_index = np.where(column_names == "city_"+city)[0][0]

In [77]:
test_array[batting_team_index] = 1
test_array[bowling_team_index] = 1
test_array[city_index] = 1
test_array[0] = runs_left
test_array[1] = balls_left
test_array[2] = wickets
test_array[3] = total_runs_x
test_array[4] = crr
test_array[5] = rrr

In [79]:
array = logistic_model.predict_proba([test_array])

c:\Users\Tushar\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [85]:
print(f"Probability of winning of {batting_team} is {array[0][1]}")
print(f"Probability of winning of {bowling_team} is {array[0][0]}")

Probability of winning of Delhi Capitals is 0.9454107445593452
Probability of winning of Chennai Super Kings is 0.05458925544065485


### **Create pickle**

In [88]:
import pickle
with open('logistic_model.pkl','wb') as f:
    pickle.dump(logistic_model,f)

In [89]:
project_data = {'columns':list(x.columns)}

### **Create json**

In [90]:
import json
with open('project_data.json','w') as f:
    json.dump(project_data,f)